#### Logistic regression

Refrencing code from

https://freedium.cfd/https://towardsdatascience.com/logistic-regression-from-scratch-in-python-ec66603592e2

https://www.analyticsvidhya.com/blog/2022/02/implementing-logistic-regression-from-scratch-using-python/

Article help from

https://arxiv.org/pdf/1609.04747.pdf

https://iamtrask.github.io/2015/07/27/python-network-part2/

https://www.kdnuggets.com/2020/05/model-evaluation-metrics-machine-learning.html



Note: epoch was set to 2000 across the board for initial testing, this causes the code to take around 60-90 seconds to run. You don't have that kind of time, so I changed them to about 800 which didn't seem to effect it much at all.

In [155]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import Normalizer
from sklearn.metrics import f1_score

# Data source: https://www.kaggle.com/datasets/luvharishkhati/heart-disease-patients-details/data
df = pd.read_csv('heart_disease.csv')

# X is the input data in the shape (m x n) aka (examples x features)
X = df.drop(columns=['result']).values
# scale the data between 0 and 1
transformer = Normalizer(norm='max').fit(X)
X = transformer.transform(X)

# y is true/target value (this can only be 0 or 1) 
y = df['result'].values



In [156]:
# Outputs a value between 0 and 1
# z is an integer equivalnet to 
# weight1*data1 + weight2*data2 + ... + weightn + datan + bias
# in the form np.dot(X, w) + b)
def sigmoid(z):
    return 1.0/(1 + np.exp(-z))

In [157]:
# Returns a value representative of the accuracy of our guesses
# y = true value (either 0 or 1)
# y_pred = probability of y being 1. Ranges from 0 to 1
# More correct predictions return closer to 0 
# Less correct predictions approach infinity

def loss(y, y_pred):
    return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

In [158]:
# Calculates the gradient
def gradients(X, y, y_pred):
   
    m = X.shape[0]
    
    # dw is the partial derivative of the loss function with respect to w 
    dw = (1/m)*np.dot(X.T, (y_pred - y))
    
    # db is the partial derivative of the Loss function with respect to b
    db = (1/m)*np.sum((y_pred - y)) 
    
    return dw, db

In [159]:
def train(X, y, bs, epochs, lr):
    
    m, n = X.shape
    
    # Initializing weights and bias to zeros
    w = np.zeros((n,1))
    b = 0
    
    losses = []

    # Reshaping y
    y = y.reshape(m,1)
    
    for epoch in range(epochs):
        # Mini batch gradient descent
        # Performs an update for every mini batch, number of mini batches
        # is defined by (m-1)//bs
        for i in range((m-1)//bs + 1):
            
            # Defining batches
            start_i = i*bs
            end_i = start_i + bs
            xb = X[start_i:end_i]
            yb = y[start_i:end_i]
            
            # Calculating hypothesis/prediction
            y_pred = sigmoid(np.dot(xb, w) + b)
            
            # Getting the gradients of loss with refrence to parameters
            dw, db = gradients(xb, yb, y_pred)
            
            # Updating the parameters
            w -= lr*dw
            b -= lr*db
        
        # Adding loss/cost to the list
        losses.append( loss(y, sigmoid(np.dot(X, w) + b)) )
        
    return w, b, losses

In [160]:
def trainSGD(X, y, epochs, lr):
    
    m, n = X.shape
    
    # Initializing weights and bias to zeros.
    w = np.zeros((n,1))
    b = 0
    
    losses = []

    # Reshaping y
    y = y.reshape(m,1)
    
    for epoch in range(epochs):
        # Stochastic Gradient Descent
        for i in range(m-1):
            
            # Calculating hypothesis/prediction
            y_pred = sigmoid(np.dot(X[i:i+1], w) + b)
            
            # Getting the gradients of loss with refrence to parameters
            dw, db = gradients(X[i:i+1], y[i], y_pred) 
            
            # Updating the parameters
            w -= lr*dw
            b -= lr*db
        
        # Calculating loss and appending it in the list.
        losses.append( loss(y, sigmoid(np.dot(X, w) + b)) )
        
    # returning weights, bias and losses(List).
    return w, b, losses

In [161]:
# Changes the y_pred range into binary predictions
def predict(X, w, b):

    # Making predictions
    preds = sigmoid(np.dot(X, w) + b)

    # if y_pred >= 0.5 round up to 1
    # if y_pred < 0.5 round down to 0
    pred_class = [1 if i > 0.5 else 0 for i in preds]
    
    return np.array(pred_class)

In [162]:
# Meaures the accuracy of the model from 0-1 (percent correct predictions)
# y is our array of target values
# (y_pred was adjusted to be ONLY 0 or 1 by the predict function, it
# is no longer a range)
def accuracy(y, y_pred):
    return np.sum(y == y_pred) / len(y)

In [163]:
# Measures correct/false positives and negatives
def accuracy_measure(y, y_pred):

    # True Positive (TP): Predicted True and True in reality.
    # True Negative (TN): Predicted False and False in reality.
    # False Positive (FP): Predicted True and False in reality.
    # False Negative (FN): Predicted False and True in reality.

    TP = np.sum(np.logical_and(y == 1, y_pred == 1))
    TN = np.sum(np.logical_and(y == 0, y_pred == 0))
    FP = np.sum(np.logical_and(y == 0, y_pred == 1))
    FN = np.sum(np.logical_and(y == 1, y_pred == 0))

    return TP,TN,FP,FN

In [164]:
# Training 
w, b, l = train(X, y, bs=10, epochs=800, lr=0.05) # Mini-batch gradient descent
w2, b2, l2 = trainSGD(X, y, epochs=800, lr=0.05) # Stochastic gradient descent

# SGD performs very marginally better under the same epochs and learning rate
# It is, however, WAY slower...

# Decreasing batch size (bs) improves the accuracy of the model but slows its training
# Increasing epochs usually improves the accuracy of the model but slows its training
# Adjusting (increasing for SGD!) learning rate (lr) may help fine tune the model

# What is our accuracy
print("Accuracy minibatch")
print(accuracy(y, y_pred=predict(X, w, b)))
print("Accuracy SGD")
print(accuracy(y, y_pred=predict(X, w2, b2)))

print("Accuracy minibatch TP,TN,FP,FN")
print(accuracy_measure(y, y_pred=predict(X, w, b)))
print("Accuracy SGD TP,TN,FP,FN")
print(accuracy_measure(y, y_pred=predict(X, w2, b2)))

print("F1 batch minibatch")
print(f1_score(y, predict(X, w, b)))
print("F1 batch SGD")
print(f1_score(y, predict(X, w2, b2)))

Accuracy minibatch
0.7074074074074074
Accuracy SGD
0.725925925925926
Accuracy minibatch TP,TN,FP,FN
(71, 120, 30, 49)
Accuracy SGD TP,TN,FP,FN
(73, 123, 27, 47)
F1 batch minibatch
0.6425339366515838
F1 batch SGD
0.6636363636363637


With the values used in the training set, bs=10, epochs=2000, lr=0.05, the accuracy of the model is about 0.72 for mini-batch and .76 for SGD. This was the best I could get after tinkering around with the values, it isn't great, but it's not terrible. Why isn't it better? After all, our data should be sufficient, it's definitely has enough to visually observe the trends in our exploratory data analysis. Plus, our data is normalized at the start of this process so there shouldn't be an issue there. 

My best guess is that the model is too complex. With 13 columns, realisitcally the model should get more time, more batches, and a more optimized learning rate to try and find the best weights and bias. I played with the epochs, bs and lr a lot but "played with" isn't exactly a watertight methodology. Either way, when I printed out the final weights in testing, they were all in the same general range. Intuitively this doesn't make sense, we can clearly see that some data trends *much* more strongly with our result than other data. The best way to deal with this would probably be to drop less relevant features to reduce noise. For example, fasting_blood_sugar trends with -0.016.

In [165]:
def trainSGD_momentum(X, y, epochs, lr, momentum):
     
    m, n = X.shape
    
    # Initializing weights, bias and velocity terms to zero
    w = np.zeros((n,1))
    b = 0
    v_w = np.zeros((n,1))
    v_b = 0
    
    # Reshaping y
    y = y.reshape(m,1)
    
    # Empty list to store losses.
    losses = []
    
    # Training loop.
    for epoch in range(epochs):
        # Stochastic Gradient Descent
        for i in range(m-1):
            
            # Calculating hypothesis/prediction
            y_pred = sigmoid(np.dot(X[i:i+1], w) + b)
            
            # Getting the gradients of loss with refrence to parameters
            dw, db = gradients(X[i:i+1], y[i], y_pred) 
            
            # Updating the parameters
            w -= lr*dw
            b -= lr*db

            # Update with momentum
            v_w = momentum * v_w + lr * dw  # Update velocity for weights
            v_b = momentum * v_b + lr * db  # Update velocity for bias
            
            # Update weights and bias
            w -= v_w
            b -= v_b
        
        # Calculating loss and appending it in the list.
        losses.append( loss(y, sigmoid(np.dot(X, w) + b)) )
        
    return w, b, losses

In [166]:
def trainSGD_Adagrad(X, y, epochs, lr):
    
    m, n = X.shape
    
    # Initializing weights, bias and accumulated square gradient to zeros
    w = np.zeros((n,1))
    b = 0
    G_w = np.zeros((n, 1))
    G_b = 0

    # For storing loss
    losses = []

    # Reshaping y
    y = y.reshape(m,1)
    
    for epoch in range(epochs):
        for i in range(m-1):
            
            # Making predictions
            y_pred = sigmoid(np.dot(X[i:i+1], w) + b)
            
            # Getting the gradients of loss with refrence to parameters
            dw, db = gradients(X[i:i+1], y[i], y_pred) 
            
            # Updating weights and bias
            w -= lr*dw
            b -= lr*db

            # Update accumulated square gradient
            G_w += dw**2
            G_b += db**2
            
            # Update weights and bias using Adagrad
            w -= (lr / (np.sqrt(G_w) + 1e-8)) * dw
            b -= (lr / (np.sqrt(G_b) + 1e-8)) * db
        
        # Adding loss/cost to the list
        losses.append(loss(y, sigmoid(np.dot(X, w) + b)))
        
    return w, b, losses

In [168]:
# Stochastic gradient descent with momentum
w, b, l = trainSGD_momentum(X, y, epochs=800, lr=0.05, momentum=0.9) 
w2, b2, l2 = trainSGD_Adagrad(X, y, epochs=800, lr=0.05) 

# What is our accuracy
print("Accuracy minibatch")
print(accuracy(y, y_pred=predict(X, w, b)))
print("Accuracy SGD")
print(accuracy(y, y_pred=predict(X, w2, b2)))

print("Accuracy minibatch TP,TN,FP,FN")
print(accuracy_measure(y, y_pred=predict(X, w, b)))
print("Accuracy SGD TP,TN,FP,FN")
print(accuracy_measure(y, y_pred=predict(X, w2, b2)))

print("F1 batch minibatch")
print(f1_score(y, predict(X, w, b)))
print("F1 batch SGD")
print(f1_score(y, predict(X, w2, b2)))


Accuracy minibatch
0.8185185185185185
Accuracy SGD
0.7407407407407407
Accuracy minibatch TP,TN,FP,FN
(92, 129, 21, 28)
Accuracy SGD TP,TN,FP,FN
(73, 127, 23, 47)
F1 batch minibatch
0.7896995708154506
F1 batch SGD
0.6759259259259259


#### Conclusion

The simple answer for why we use the gradient descent improvements is because they are improvements. The accuracy of the model is better though the time it takes to run increases somewhat. As often is the case, it's a balancing act, is that extra training time worth the increase in model accuracy. In our case, ~15 seconds increase is worth it because 15 seconds isn't that much. Proportionally that is a lot however, so be mindful on larger data sets and weaker computers. (Note that most of this increase in time comes from the fact that both of the optimizations are perfomred on SGD which takes WAY longer than mini batch.)

In this case, Adagrad seems to be a more marginal improvement while momentum makes the model almost a full 8% better! Given the data we are working with, this is likely because it is so noisy, see EDA for a visual refresher. The data is also "weird" since many of the features are quantized. Adagrad can be sensitive to noise and situations where the landscape does weird stuff like just from 3 to 6 to 7 exclusively. Since the historical gradient may not be the most informative if it's jumping around like a bunch of V's. Momentum on the other hand is suited to escape areas of weird noise and won't suffer from trying to adapt to historical gradient.

For measuring accuracy, the counts of True Positive, True Negative, False Positive and False Negative were included. This works for two reasons, one it helps inform the threshold, if your false positive is really high and your false negative is really low, it would be a good idea to move the threshold down and vice versa. Second, it informs our use of the F1 score as a second method of measurement.

"The F1 score is the harmonic mean of the precision and recall, where an F1 score reaches its best value at 1 (perfect precision and recall) and worst at 0.

An F1 score punishes extreme values more. Ideally, an F1 Score could be an effective evaluation metric in the following classification scenarios:

When FP and FN are equally costly
Adding more data doesn’t effectively change the outcome effectively
TN is high (like with flood predictions, cancer predictions, etc.) "

This is perfect! We fit all of these criteria. Our FN and FP values are about equal and TN is high. Apparently this is common in medical models due to its effectiveness in handling imbalanced datasets, a common characteristic in medical data where one class (e.g., disease presence) may be significantly less frequent than the other. F1 score considers both precision and recall, so it's for situations where false positives and false negatives carry different clinical implications. You don't want to end up being told you have heart disease when you don't! (Though the other way around would be worse...) The F1 score is ideal for measuing the quality of models that assist doctors in making decisions, perfect for this example.







